In [1]:
import os
import datetime
import google.generativeai as genai
from dotenv import load_dotenv
from tqdm.notebook import tqdm
import time
import logging

logger = logging.getLogger(__name__)

def localtime_to_str():
    return time.strftime('%Y-%m-%d_%H-%M-%S', time.localtime())

logging.basicConfig(
    filename=f"data/output/gemini/log/gemini_{localtime_to_str()}.log", 
    encoding="utf-8",
    level=logging.DEBUG
    )

load_dotenv()

genai.configure(api_key=os.environ["GEMINI_API_KEY"])

def create_dir(base_dir="data/output/gemini/responses") -> str:
    """
    指定したベースディレクトリに、日付と連番を組み合わせた名称のディレクトリを作成する。
    既に同名のディレクトリが存在する場合、連番をインクリメントして新たなディレクトリを作成する。

    Args:
        base_dir: ベースとなるディレクトリのパス (デフォルト: "data/output/gemini/responses")

    Returns:
        str: 作成されたディレクトリの相対パス
    """

    today = datetime.date.today().strftime("%Y%m%d")
    num = 1

    while True:
        dir_name = f"{base_dir}/{today}_{num:03d}"  # 連番を3桁で表示
        try:
            if not os.path.exists(dir_name):
                os.makedirs(dir_name)
                return dir_name
            num += 1
        except OSError as e:
            print(f"ディレクトリ作成中にエラーが発生しました: {e}")

In [2]:
# Create the model
generation_config = {
    "temperature": 1,
    "top_p": 0.95,
    "top_k": 40,
    "max_output_tokens": 8192,
    "response_mime_type": "text/plain",
}

model = genai.GenerativeModel(
    model_name="gemini-2.0-flash-exp",
    generation_config=generation_config,
)

chat_session = model.start_chat(history=[])

In [3]:
prompt_cause_of_the_incident = """
### 指示 ###
あなたは製品の事故原因を分析しています。
事故の内容から、事故の原因を単純に説明してください。
回答形式はJSON形式です。

### 回答形式 ###
{
	"原因":"回答"
}

### 注意事項 ###
- 指示された回答のみ出力してください。
- 事故の原因を最もよく説明する単語のみで回答してください。
- 原因は複数回答してもかまいません。その場合、配列で記述してください。
- 解説は不要です。

### 例 ###
事故の内容:
五徳付プレートの外側の縁が処理されておらず、多少ざらついていることから、素手でプレートを洗っていた際に、外側の縁に指を強く擦りつけたため、負傷したものと推定される。なお、取扱説明書には「手、指の保護のため、必ずゴム手袋などを使用してください。」との注意が記載されている。

回答:
{
	"原因":["縁","接触"]
}

### 例の解説 ###
「縁に指を強く擦りつけた」という記述から、「縁」、「接触」が原因であると判断できます。

### 事故の内容 ###
"""

In [4]:
import pandas as pd
import json

In [5]:
global former_data
former_data = ""

In [6]:
def generate_json_response(
        prompt:str, 
        data:pd.DataFrame, 
        response_name:str, 
        chat_session:genai.ChatSession,
        request_interval:int=6
        ) -> None:
    global former_data
    for i in tqdm(range(0, len(data))):
        if data.values[i] == former_data:
            logger.debug(f"Skipped: {data.values[i]}")
            data.values[i] = data.values[i-1]
            continue
        input_prompt = prompt + data.values[i]
        former_data = data.values[i]
        try:
            response = chat_session.send_message(input_prompt)
            temp = response.text
            if "```json" in temp:
                temp = temp.replace("```json", "")
                temp = temp.replace("```", "")
            res = json.loads(temp)
            logger.debug(temp)
        except Exception as e:
            logger.error("Error", e, temp)
            continue
        output_text = ""
        if isinstance(res[response_name], list):
            for text in res[response_name]:
                output_text += text + " "
        else:
            output_text = res[response_name]
        logger.debug(f"Output: {output_text}")
        data.values[i] = output_text
        time.sleep(request_interval)

In [7]:
import settings as st

output_dir = create_dir()

In [8]:
def processor(product:str)->None:
    data = pd.read_csv(f"data/output/gemini/prompt_data/{product}.csv", encoding="utf-8-sig")
    generate_json_response(
        prompt=prompt_cause_of_the_incident, 
        data=data["事故原因"], 
        response_name="原因", 
        chat_session=chat_session
        )
    print(data.head())
    data = data.drop(columns=["事故通知内容"])
    data.to_csv(f"{output_dir}/{product}_res.csv", encoding="utf-8-sig", index=False)

In [9]:
for product in st.products_250115:
    processor(product)

  0%|          | 0/170 [00:00<?, ?it/s]

     No                                             事故通知内容          事故原因
0   584  当該オーブントースターで魚のホイル焼きをしていると、本体が異常に熱くなったのに気付き、電源プ...     設定 温度 耐熱 
1  1502                  オーブントースターを使用中、上部ヒーター付近の金属板から発火した。      餅 過熱 付着 
2  1975  オープントースターで食パンを焼いていたら、パンに火がつき、炎が本体扉の隙間から外へ10センチ...  タイマー 誤設定 過熱 
3  3852           使用中の電気オーブントースターから出火し、周辺を焼損した。(事故発生地:兵庫県)  油 落下 過熱 トレー 
4  4017       ネット通販で購入したオーブントースターを使用中、庫内から火が出た。(事故発生地:埼玉県)  油脂 堆積 過熱 汚れ 


  0%|          | 0/341 [00:00<?, ?it/s]

   No                                      事故通知内容                      事故原因
0  94  おもちゃのパトカーに使用していたアルカリ乾電池が液漏れし、タオルケット等に付着した。  プラスチックシール 傾き ストレス 割れ 漏液 
1  95             使用していた乾電池(アルカリ)の液漏れにより、玩具を汚損した。  プラスチックシール 傾き ストレス 割れ 漏液 
2  96             時計に使用していたアルカリ乾電池が液漏れし、ケースに付着した。  プラスチックシール 傾き ストレス 割れ 漏液 
3  97       使用していた乾電池(アルカリ)の液漏れにより、リモコンが使用不能となった。  プラスチックシール 傾き ストレス 割れ 漏液 
4  98    カメラに使用していた乾電池(アルカリ)の液漏れにより、カメラが使用不能になった。  プラスチックシール 傾き ストレス 割れ 漏液 


  0%|          | 0/247 [00:00<?, ?it/s]

...

  0%|          | 0/171 [00:00<?, ?it/s]

    No                                             事故通知内容                事故原因
0   43  1年前に購入した脚立(園芸用三脚)に登ろうとしたところ、突然1脚側支柱が折れ、脚立が倒れたた...   バランス 転倒 荷重 支柱 破損 
1   45  脚立(園芸用三脚)の中段にのって自宅壁の塗装補修作業中、突然片側支柱が曲がって、脚立が倒れた...   支柱 軟弱 地面 転倒 バランス 
2  116  庭木のせん定に使用する3本足の脚立に乗り、軒先に吊してあったすだれを外そうとしたとき、突然転...   バランス 転倒 応力 支柱 折損 
3  333  配線作業のため、天板をまたいで天板直下の踏みざんに乗って作業中、突然支柱が内側に曲がったため...  バランス 転倒 支柱 接触 曲がり 
4  569  脚立の天板をまたぐように下から3段目の踏みざんに乗って作業をしていたところ、突然、両脚が曲が...  バランス 転倒 落下 支柱 曲がり 


  0%|          | 0/53 [00:00<?, ?it/s]

--- Logging error ---
Traceback (most recent call last):
  File "C:\Users\chika\AppData\Local\Temp\ipykernel_22912\325136422.py", line 17, in generate_json_response
    response = chat_session.send_message(input_prompt)
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\chika\Documents\VSCode\research-utils\.venv\Lib\site-packages\google\generativeai\generative_models.py", line 578, in send_message
    response = self.model.generate_content(
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\chika\Documents\VSCode\research-utils\.venv\Lib\site-packages\google\generativeai\generative_models.py", line 331, in generate_content
    response = self._client.generate_content(
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\chika\Documents\VSCode\research-utils\.venv\Lib\site-packages\google\ai\generativelanguage_v1beta\services\generative_service\client.py", line 830, in generate_content
    response = rpc(
               ^^^^
  File "c:\Users\chi

     No                                             事故通知内容  \
0  1043            事務所で当該製品で細断中、左手指が巻き込まれ、負傷した。(事故発生地:東京都)   
1  1640  可燃性ガスを含む潤滑剤スプレーを使用して当該製品を整備中、当該製品を焼損する火災が発生した。...   
2  2410  当該製品でCD(コンパクトディスク)を細断中、CD中央部の穴に入れていた指を穴から抜く際に、...   
3  2487  当該製品の細断部のみを本体から外して使用し、紙詰まりをそのままにしていたところ、当該製品の細...   
4  2693  当該製品にエアゾール缶のガスを吹き付け清掃を行い、その後、当該製品を使用したところ、当該製品...   

                                                事故原因  
0                                 用紙投入口 小冊子 押し込み 接触   
1  ○当該製品に焼損等の異常は認められず、正常に動作した。○紙投入口に付着した液体から、可燃性ガ...  
2                                 CD 穴 指 挟まる 裁断 投入口   
3                               エアースプレー 火花 引火 分解 改造   
4                                 エアゾール 可燃性ガス 引火 火花   
